# Script for generating chromosome matrix from GRCh37 and dbSNP

In [1]:
import os, sys
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import pysam
import time
import tqdm

# Load FASTA & Tabix

In [3]:
input_fasta_path = './raw_data/hs37d5.fa' # Reference Fasta File
input_tabix_path = './raw_data/GCF_000001405.25.gz' # dbSNP Tabix File
output_path = './pretraining_data' # Output Folder Path

In [4]:
def parse_fasta(path, header_identifier):
    # Read file
    lines = open(path,'r').readlines()
    lines = list(map(lambda x: x[:-1], lines)) # remove \n
    
    # Concatenate sequence data per chromosome
    chromosome_dict = {}
    chromosome_name = None
    chromosome_sequence = ''
    for line in lines:
        if header_identifier in line:
            if chromosome_name:
                chromosome_dict[chromosome_name] = ''.join(chromosome_sequence)
            # Reinitialize name & sequence
            chromosome_name = line[:-1]        
            chromosome_sequence = []
        else:
            # Fill Sequence
            chromosome_sequence.append(line)
            
    # Add the last chromosome
    if chromosome_name:
        chromosome_dict[chromosome_name] = ''.join(chromosome_sequence)
        
    return chromosome_dict

In [5]:
%%time
c37_dict = parse_fasta(input_fasta_path,'>')
c37_lens = [len(seq) for seq in c37_dict.values()]
print('Total Length GRch37', sum(c37_lens))
print('Min Length GRch37', min(c37_lens))
print('Max Length GRch37', max(c37_lens))

Total Length GRch37 3137454505
Min Length GRch37 4262
Max Length GRch37 249250621
CPU times: user 19.6 s, sys: 8.04 s, total: 27.6 s
Wall time: 29.1 s


In [7]:
tb_file = pysam.TabixFile(input_tabix_path)

# Filter only chromosome contigs for FASTA and tabix

In [8]:
fasta_chr_keyword = 'dna:chromosome'
fasta_chr_keys = [k for k in c37_dict.keys() if fasta_chr_keyword in k]

In [9]:
tb_chr_keyword = 'NC_0000'
tabix_chr_keys = [k for k in tb_file.contigs if tb_chr_keyword in k]

In [10]:
chr_lens = []
for fasta_chr_key in fasta_chr_keys:
    chr_lens.append(len(c37_dict[fasta_chr_key]))

# Construct the chromosome matrix for each chromosome

In [13]:
def parse_freq(freq_str):
    proba_list = []
    for src_proba_str in freq_str.split('|'):
        probas = src_proba_str.split(':')[-1].split(',')
        proba_list.append([0.0 if proba == '.' else float(proba) for proba in probas])
        
    # Calculate proba
    probas = np.array(proba_list).mean(axis=0)    
    return probas
    
def parse_vcf_row(vcf_str):
    # parse vcf row
    cols = vcf_str.split('\t')
    snv_pos, ref, alts = int(cols[1]) - 1, cols[3], np.array([cols[3]] + cols[4].split(','))
    
    # retrieve probas from freq info
    freq_str = cols[-1].split('FREQ=')[-1].split(';')[0]
    probas = parse_freq(freq_str)
    
    # filter ref_alt and probas
    alts, probas = alts[probas > 0], probas[probas > 0]
    
    return snv_pos, snv_pos + len(ref) - 1, ref, alts, probas

# Nucleotide to index (https://en.wikipedia.org/wiki/FASTA_format#Sequence_representation)
nt_to_index = { 
    'A': [0], 'G': [1], 'T': [2], 'C': [3], 'N': [4], 'DEL': [5], 'AI': [6], 'GI': [7], 'TI': [8], 'CI': [9], 'NI': [10],
    'U': [2], 'R': [0, 1], 'Y': [2, 3], 'K': [1, 2], 'M': [0, 3], 'S': [1, 3], 'W': [0, 2],
    'UI': [8], 'RI': [6, 7], 'YI': [8, 9], 'KI': [7, 8], 'MI': [6, 9], 'SI': [7, 9], 'WI': [6, 8],
    'B': [1, 2, 3], 'H': [0, 2, 3], 'V': [0, 1, 3], 'D': [0, 1, 2],
    'BI': [7, 8, 9], 'HI': [6, 8, 9], 'VI': [6, 7, 9], 'DI': [6, 7, 8]
}

In [15]:
%%time
# Loop over Chromosome
batch_len = 100000
for fasta_chr_key, tabix_chr_key, chr_len in zip(fasta_chr_keys, tabix_chr_keys, chr_lens):
    start_time = time.time()    
    
    fasta_seq = c37_dict[fasta_chr_key]  # Fasta reference
    data_block = np.zeros((chr_len, 11))  # Target block
    
    num_batch = (chr_len // batch_len) + 1
    last_next_pos = 0
    for nb in tqdm.tqdm(range(num_batch)):
        start_query_pos, end_query_pos = nb * batch_len, (nb + 1) * batch_len
        for row in tb_file.fetch(tabix_chr_key, start_query_pos, end_query_pos):
            if ';COMMON' in row:
                # parse vcf row
                start_pos, end_pos, ref, alts, probas = parse_vcf_row(row)

                # handle SNV and INDEL
                for alt, proba in zip(alts, probas):
                    if len(alt) == len(ref):
                        # SNV / MUTATION - assign proba to mutated nucleotide
                        for i, nt in enumerate(alt):
                            if nt is not 'N':
                                data_block[start_pos + i, nt_to_index[nt]] += proba / len(nt_to_index[nt])
                    elif len(alt) < len(ref):
                        # DELETION
                        # assign proba to non-deleted prefix
                        for i, nt in enumerate(alt):
                            data_block[start_pos + i, nt_to_index[nt]] += proba / len(nt_to_index[nt])

                        # assign proba to deleted suffix
                        for i in range(len(alt), len(ref)):
                            data_block[start_pos + i, nt_to_index['DEL']] += proba / len(nt_to_index['DEL'])
                            
                    else: # if len(alt) > len(ref):
                        # INSERTION
                        # assign proba to the prefix nucleotide
                        for i in range(len(ref) - 1):
                            data_block[start_pos + i, nt_to_index[alt[i]]] += proba / len(nt_to_index[alt[i]])
                            
                        # assign insertion proba to the last nucleotide
                        data_block[start_pos+len(ref)-1, nt_to_index[f'{alt[len(ref)-1]}I']] += proba / len(nt_to_index[f'{alt[len(ref)-1]}I'])

                # assign reference to region between SNV
                for i in range(last_next_pos, start_pos):
                    fasta_nt = fasta_seq[i]
                    data_block[i, nt_to_index[fasta_nt]] = 1 / len(nt_to_index[fasta_nt])

                # assign new last next position
                last_next_pos = start_pos + len(ref)

    # assign from last next position onward with nucleotide from reference
    for i in range(last_next_pos, chr_len):
        fasta_nt = fasta_seq[i]
        data_block[i, nt_to_index[fasta_nt]] = 1 / len(nt_to_index[fasta_nt])
    
    # Normalize over 11D
    data_block = data_block / data_block.sum(axis=1, keepdims=True)
    
    # Dump chromosome data    
    np.save(f'{output_path}/{tabix_chr_key}.npy', data_block)
    print(f'Finish processing chromosome {tabix_chr_key} | Elapsed time : {time.time() - start_time}s')

100%|██████████| 2493/2493 [14:45<00:00,  2.82it/s]


Finish processing chromosome NC_000001.10 | Elapsed time : 1047.1865592002869s


100%|██████████| 2432/2432 [14:29<00:00,  2.80it/s]


Finish processing chromosome NC_000002.11 | Elapsed time : 992.5833151340485s


  0%|          | 0/1912 [00:00<?, ?it/s]

...

Finish processing chromosome NC_000022.10 | Elapsed time : 195.44876289367676s


  0%|          | 0/594 [00:00<?, ?it/s]

Finish processing chromosome NC_000023.10 | Elapsed time : 613.399749994278s


100%|██████████| 594/594 [02:49<00:00,  3.50it/s]


Finish processing chromosome NC_000024.9 | Elapsed time : 192.52996969223022s
CPU times: user 3h 4min 47s, sys: 14min 48s, total: 3h 19min 35s
Wall time: 3h 41min 12s
